In [36]:
import os
import json
import pandas as pd
import pytz
from datetime import datetime
from sqlalchemy import create_engine


In [37]:
# 접속 정보
host = "10.28.224.177"
port = 30634
database = "postgres"
username = "postgres"
password = "0104"

In [ ]:
# SQLAlchemy 엔진 생성
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}")

# 유튜브 데이터 DB에 저장

### channel

In [12]:
channel_lst = ["피식대학", "SGBG", "swab85", "konuri", "Yoiki", "NERDULT", "donghahahaha", "JBKWAK", "geean84", "carthejungwon"]
channel_name = ['피식대학', '싱긍벙글', '승우아빠', '고누리', '가요이', '너덜트', '동하하', '곽튜브', '기안84', '카더정원']

In [13]:
columns_name = ['id', 'name' , 'uId', 'title', 'description', 'publishedAt', 'thumbnails', 'viewCount', 'subscriberCount', 'videoCount', 'topicCategories', 'dataTime']

In [21]:
# 한국 시간대 설정
korea_timezone = pytz.timezone('Asia/Seoul')

# 현재 시간 가져오기
korea_time = datetime.now(korea_timezone)

# 원하는 형식으로 출력 (YYYY-MM-DD)
formatted_time = korea_time.strftime('%Y-%m-%d')

In [23]:
result = []
for name, channel in zip(channel_name, channel_lst):
    with open('./'+ channel + '/' + 'chanel_info.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
    print(data)
    items = data['items']
    uId = items[0]['id']
    title = items[0]['snippet']['title']
    description = items[0]['snippet']['description']
    publishedAt = items[0]['snippet']['publishedAt']
    thumbnails = items[0]['snippet']['thumbnails']['default']['url']
    viewCount = items[0]['statistics']['viewCount']
    subscriberCount = items[0]['statistics']['subscriberCount']
    videoCount = items[0]['statistics']['videoCount']
    topicCategories = items[0]['topicDetails']['topicCategories']
    dataTime = formatted_time
    result.append([name, uId, title, description, publishedAt, thumbnails, viewCount, subscriberCount, videoCount, topicCategories, dataTime])

{'kind': 'youtube#channelListResponse', 'etag': 'LXVXij--1XO-PEvmvyJd-aHX-Uc', 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5}, 'items': [{'kind': 'youtube#channel', 'etag': 'X0MC8wQC67nrc7tnpHRGlbgScQQ', 'id': 'UCGX5sP4ehBkihHwt5bs5wvg', 'snippet': {'title': '피식대학Psick Univ', 'description': '코미디 인재 육성 및 연구의 메카 피식대학 \n\n✅ 피식대학 비즈니스 메일\npsickuniv@metacomedy.net\n\n✅ 피식대학 사서함\n(04054) 서울특별시 마포구 와우산로 29가길 42, 메타코미디\n\n✅ 피식대학 굿즈 스토어\nhttps://smartstore.naver.com/psickuniv\n', 'customUrl': '@피식대학', 'publishedAt': '2019-04-01T05:31:41Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1OnV8wL-9IzC4aRr52M6q4z8I=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1OnV8wL-9IzC4aRr52M6q4z8I=s240-c-k-c0x00ffffff-no-rj', 'width': 240, 'height': 240}, 'high': {'url': 'https://yt3.ggpht.com/FQ61Pjh2In6lt-nd0o6EtWyCRPIlopBzcm-Pz88CzRXE1gZQ-o1On

In [26]:
df_c = pd.DataFrame(result)

In [29]:
df_c = df_c.reset_index()
df_c.columns = columns_name

In [31]:
df_c.to_csv('channel_v1.csv', index=False)

In [41]:
table_name = "Channel"  # 저장할 테이블 이름
df_c.to_sql(table_name, engine, if_exists="replace", index=False)

10

### Video

In [82]:
columns_name = ['channel_id', 'vId','videoTitle', 'videoDescription', 'videoPublishedAt', 'videoThumbnails', 'tags', 'categoryId', 'liveBroadcastContent', 'videoViewCount', 'videoLikeCount', 'videoDislikeCount', 'commentCount', 'topicDetails']

In [83]:
result = []
for channel_id, channel in zip(range(len(channel_lst)), channel_lst):
    files = os.listdir(channel)
    files = [x for x in files if x.find('video_') != -1]
    for f in files:
        with open('./'+ channel + '/' + f, 'r', encoding='utf-8') as file:
            data = json.load(file)
        # print(data)
        itmes = data['items']
        vId = itmes[0]['id']
        videoTitle = itmes[0]['snippet']['title']
        videoDescription = itmes[0]['snippet']['description']
        videoPublishedAt = itmes[0]['snippet']['publishedAt']
        videoThumbnails = itmes[0]['snippet']['thumbnails']['default']['url']
        tags = itmes[0]['snippet'].get('tags')
        categoryId = itmes[0]['snippet']['categoryId']
        liveBroadcastContent = itmes[0]['snippet']['liveBroadcastContent']
        
        videoViewCount = itmes[0]['statistics']['viewCount']
        videoLikeCount = itmes[0]['statistics'].get('likeCount')
        # print(videoLikeCount)
        videoDislikeCount = itmes[0]['statistics'].get('dislikeCount')
        commentCount = itmes[0]['statistics'].get('commentCount')
        topicDetails = itmes[0].get('topicDetails')
        if topicDetails:
            topicDetails = topicDetails['topicCategories']
            
        result.append([channel_id, vId, videoTitle, videoDescription, videoPublishedAt, videoThumbnails, tags,
                       categoryId, liveBroadcastContent, videoViewCount, videoLikeCount, 
                       videoDislikeCount, commentCount, topicDetails])
    

In [84]:
df_v = pd.DataFrame(result, columns=columns_name)

In [88]:
df_v = df_v.sort_values(['channel_id', 'videoPublishedAt'])

In [91]:
len(df_v)

2191

In [89]:
df_v.head(3)

,channel_id,vId,videoTitle,videoDescription,videoPublishedAt,videoThumbnails,tags,categoryId,liveBroadcastContent,videoViewCount,videoLikeCount,videoDislikeCount,commentCount,topicDetails
161,0,yRHHNfkCcoM,김연아씨에게 할 말이 있습니다,#김연아 #피식대학 #정재형,2022-07-25T06:42:57Z,https://i.ytimg.com/vi/yRHHNfkCcoM/default.jpg,None,23,none,702484,11255,None,939,None
117,0,fcJP82byDZc,(중년커버) A Whole New World - Aladdin OST (알라딘) C...,#한사랑산악회 #배용길 #나점례 #awholenewworld #aladdin,2022-07-25T11:15:13Z,https://i.ytimg.com/vi/fcJP82byDZc/default.jpg,None,23,none,237070,4691,None,608,[https://en.wikipedia.org/wiki/Music]
228,0,bW_aXDkr6kw,그동안 영상을 못 올린 이유...,#피식대학 #로니앤스티브 #폐업,2022-07-29T12:00:11Z,https://i.ytimg.com/vi/bW_aXDkr6kw/default.jpg,None,23,none,558310,6202,None,480,[https://en.wikipedia.org/wiki/Lifestyle_(soci...


In [90]:
df_v.to_sql('Video', engine, if_exists="replace", index=False)

191

### Comments

In [101]:
columns_name = ['vId', 'cId', 'textDisplay', 'authorDisplayName', 'likeCount', 'commentPublishedAt', 'commentUpdatedAt', 'replyCount', 'replies']

In [100]:
result = []
for channel_id, channel in zip(range(len(channel_lst)), channel_lst):
    files = os.listdir(channel)
    files = [x for x in files if x.find('comment_') != -1]
    for f in files:
        with open('./'+ channel + '/' + f, 'r', encoding='utf-8') as file:
            data = json.load(file)

        itmes = data['items']
        vId = itmes[0]['snippet']['videoId']
        cId = itmes[0]['id']
        
        textDisplay = itmes[0]['snippet']['topLevelComment']['snippet']['textDisplay']
        authorDisplayName = itmes[0]['snippet']['topLevelComment']['snippet']['authorDisplayName']
        likeCount = itmes[0]['snippet']['topLevelComment']['snippet']['likeCount']
        commentPublishedAt = itmes[0]['snippet']['topLevelComment']['snippet']['publishedAt']
        commentUpdatedAt = itmes[0]['snippet']['topLevelComment']['snippet']['updatedAt']
        replyCount = itmes[0]['snippet']['totalReplyCount']
        
        replies_lst = itmes[0].get('replies')
        if replies_lst:
            replies = [x['snippet']['textDisplay'] for x in replies_lst['comments']]
            
        result.append([vId, cId, textDisplay, authorDisplayName, likeCount, commentPublishedAt, commentUpdatedAt, replyCount, replies])
    

In [102]:
df_co = pd.DataFrame(result, columns=columns_name)

In [107]:
len(df_co)

6092

In [106]:
df_co.head(3)

,vId,cId,textDisplay,authorDisplayName,likeCount,commentPublishedAt,commentUpdatedAt,replyCount,replies
0,hBgp-u5aEtE,UgyXRQTTnO45AQfIPjV4AaABAg,길은지의 닉네임을 정해주세요<br>1.옥순<br>2.영자<br>3.영숙<br>4.영...,@피식대학,1938,2023-02-11T12:21:00Z,2023-02-11T12:26:37Z,466,"[간략히, 1, 춘자 ㄱ, 옥순 한 표용, 2번도 영자고 4번도 영자네요...? 전..."
1,EkCY92AlzqU,UgyN5CT08NW28_-gvAR4AaABAg,아콰피나 무슨 홍보 때문에 나온게 아니라 진짜 한국에 놀러 왔다가 피식쇼에 출연한게...,@9birdwatching,5735,2024-02-25T10:58:53Z,2024-02-25T10:58:53Z,9,[아마 엘에이 교포 친구들이 소개해준듯요 ㅋㅋ 아콰피나 파티에서 만난적 있는데 엄첨...
2,fhxzI81iDUA,UgxDsePGUwEQEODBl6F4AaABAg,좋아요 안누르면 반동분자로 간주하거쏘. <br>-레릴리,@aileemusic,14491,2023-10-01T11:18:33Z,2023-10-01T11:18:33Z,97,"[레릴리 이쁜줄은 알았는데 오늘은 겁나이쁘다!!, You have become pr..."


In [104]:
df_co.to_sql('Comments', engine, if_exists="replace", index=False)

92

In [115]:
query = f'SELECT * FROM public."Comments"'
comments_df = pd.read_sql(query, engine)

In [116]:
comments_df

,vId,cId,textDisplay,authorDisplayName,likeCount,commentPublishedAt,commentUpdatedAt,replyCount,replies
0,hBgp-u5aEtE,UgyXRQTTnO45AQfIPjV4AaABAg,길은지의 닉네임을 정해주세요<br>1.옥순<br>2.영자<br>3.영숙<br>4.영...,@피식대학,1938,2023-02-11T12:21:00Z,2023-02-11T12:26:37Z,466,"{간략히,1,""춘자 ㄱ"",""옥순 한 표용"",""2번도 영자고 4번도 영자네요...? ..."
1,EkCY92AlzqU,UgyN5CT08NW28_-gvAR4AaABAg,아콰피나 무슨 홍보 때문에 나온게 아니라 진짜 한국에 놀러 왔다가 피식쇼에 출연한게...,@9birdwatching,5735,2024-02-25T10:58:53Z,2024-02-25T10:58:53Z,9,"{""아마 엘에이 교포 친구들이 소개해준듯요 ㅋㅋ 아콰피나 파티에서 만난적 있는데 엄..."
2,fhxzI81iDUA,UgxDsePGUwEQEODBl6F4AaABAg,좋아요 안누르면 반동분자로 간주하거쏘. <br>-레릴리,@aileemusic,14491,2023-10-01T11:18:33Z,2023-10-01T11:18:33Z,97,"{""레릴리 이쁜줄은 알았는데 오늘은 겁나이쁘다!!"",""You have become ..."
3,MyhnfHQJUWk,Ugwwr6CZIvazgoTzQr14AaABAg,그냥 누구세요 부터 끝났잖앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,@vo2ce251,2437,2024-12-21T10:11:56Z,2024-12-21T10:11:56Z,9,"{""ㅋㅋ 마니 긁혔어 ㅋㅋㅋ 첨지 춘봉앜빠 승"",""솔직히 영우가 동나이대에서 제일 ..."
4,I7yaklZTSiQ,UgxSAQXWomyZ2Qfl-L14AaABAg,재형이형님 눈썹 미셨나,@Halloweencandy.,2,2024-12-09T05:27:46Z,2024-12-09T05:27:46Z,0,"{""ㅋㅋ 마니 긁혔어 ㅋㅋㅋ 첨지 춘봉앜빠 승"",""솔직히 영우가 동나이대에서 제일 ..."
...,...,...,...,...,...,...,...,...,...
6087,xxhiOsXgJ8c,UgwPst6zkXPsef4aEol4AaABAg,귀한분 모셔놓고 습관성ㅋㅋㅋ<br>이준혁❤❤,@PJE0818,3,2025-01-09T10:57:03Z,2025-01-09T10:57:03Z,0,"{심심하네요,""솔직히 누가 남고에 장기하 님을 부를 거라고 예상했을까<br>영상 보..."
6088,ViORd1jnaVA,Ugw5_XWZ2H2_kzUBM0V4AaABAg,개꾸릉네님 비흡연자인 저도 사고싶어지네요,@oUo-_-oUo,0,2024-03-27T03:20:27Z,2024-03-27T03:20:27Z,0,"{심심하네요,""솔직히 누가 남고에 장기하 님을 부를 거라고 예상했을까<br>영상 보..."
6089,j5JzbcivCcw,UgwaAy_jXcITHWmyi1R4AaABAg,역시 연기자는 다르네 ㅎㅎ진짜 맛깔지게 잘하심ㅎ 의성배우님 서윗하심,@momor243,29,2024-02-10T19:37:42Z,2024-02-10T19:37:42Z,0,"{심심하네요,""솔직히 누가 남고에 장기하 님을 부를 거라고 예상했을까<br>영상 보..."
6090,AZG2P3JjYoI,Ugx6pxbz9dM1sa35_y14AaABAg,고등학교 봉사활동으로 선플달기 활동중입니다 카더정원 유튜브 항상 잘 보고 있어요 오...,@팬이아닌아저씨는-n5j,3,2024-11-22T04:29:32Z,2024-11-22T04:29:32Z,0,"{심심하네요,""솔직히 누가 남고에 장기하 님을 부를 거라고 예상했을까<br>영상 보..."
